# Tutorial 16: Subgraph Patterns

In this tutorial, you'll learn to build **composable, reusable graph components** using subgraphs.

**What you'll learn:**
- Wrapping subgraphs as nodes in parent graphs
- State transformation between parent and subgraph
- Creating reusable graph components
- Chaining, conditional, and parallel subgraph patterns
- Building modular agent systems

By the end, you'll be able to compose complex workflows from simple, tested building blocks.

## Prerequisites

- Completed Tutorials 14-15 (Multi-Agent Patterns)
- Understanding of StateGraph and state management
- Familiarity with TypedDict and type transformations

## Why Subgraphs?

As agent systems grow, you need:

1. **Reusability**: Use the same RAG pipeline in multiple agents
2. **Encapsulation**: Hide internal complexity behind clean interfaces
3. **Testing**: Test components independently before composing
4. **Modularity**: Swap implementations without changing the parent graph

```
┌─────────────────────────────────────────────────────────────┐
│                      Parent Graph                          │
│                                                             │
│  ┌─────────┐    ┌─────────────────────────┐    ┌─────────┐ │
│  │  Entry  │───▶│      Subgraph           │───▶│  Next   │ │
│  │  Node   │    │  (Black Box)            │    │  Node   │ │
│  └─────────┘    └─────────────────────────┘    └─────────┘ │
│                           │                                 │
│              State In ────┴──── State Out                   │
└─────────────────────────────────────────────────────────────┘
```

The key insight: **subgraphs can have different state schemas** than the parent graph. State transformation functions bridge the gap.

## Step 1: Setup

In [ ]:
from langgraph_ollama_local import LocalAgentConfig
from langchain_ollama import ChatOllama

config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
)

print(f"Model: {config.ollama.model}")
print("Setup complete!")

## Step 2: Create a Simple Subgraph

Let's create a simple summarization subgraph that we'll embed in a larger system.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage


# Subgraph has its own state schema
class SummaryState(TypedDict):
    """State for the summarization subgraph."""
    text: str
    summary: str
    word_count: int


def summarize_node(state: SummaryState) -> dict:
    """Summarize the input text."""
    response = llm.invoke([
        SystemMessage(content="Summarize the following text in 2-3 sentences."),
        HumanMessage(content=state["text"]),
    ])
    summary = response.content
    return {
        "summary": summary,
        "word_count": len(summary.split()),
    }


# Build the subgraph
summary_workflow = StateGraph(SummaryState)
summary_workflow.add_node("summarize", summarize_node)
summary_workflow.add_edge(START, "summarize")
summary_workflow.add_edge("summarize", END)

summary_graph = summary_workflow.compile()

print("Summary subgraph created!")

# Test it standalone
result = summary_graph.invoke({
    "text": "Python is a high-level programming language known for its simplicity and readability. It supports multiple programming paradigms including procedural, object-oriented, and functional programming. Python has a large standard library and active community.",
    "summary": "",
    "word_count": 0,
})

print(f"\nSummary: {result['summary']}")
print(f"Word count: {result['word_count']}")

## Step 3: Create a Parent Graph with Different State

Our parent graph has a different state schema. We need to transform between them.

In [ ]:
# Parent graph has a different state schema
class DocumentState(TypedDict):
    """State for document processing."""
    messages: Annotated[list, add_messages]
    document_content: str
    document_summary: str
    processed: bool


print("Parent state schema:")
print("  - messages: conversation history")
print("  - document_content: the raw document")
print("  - document_summary: will hold the summary")
print("  - processed: flag when complete")
print("\nSubgraph state schema:")
print("  - text: input text")
print("  - summary: output summary")
print("  - word_count: summary length")

## Step 4: Define State Transformation Functions

The key to subgraph composition: **state_in** and **state_out** functions.

In [ ]:
def summary_state_in(parent_state: DocumentState) -> SummaryState:
    """
    Transform parent state to subgraph input.
    
    Maps: document_content -> text
    """
    return {
        "text": parent_state["document_content"],
        "summary": "",
        "word_count": 0,
    }


def summary_state_out(subgraph_state: SummaryState, parent_state: DocumentState) -> dict:
    """
    Transform subgraph output to parent state updates.
    
    Maps: summary -> document_summary
    """
    return {
        "document_summary": subgraph_state["summary"],
    }


print("State transformers defined!")
print("\nstate_in: DocumentState -> SummaryState")
print("state_out: SummaryState -> dict (parent updates)")

## Step 5: Wrap Subgraph as Node

Now we create a node function that wraps the subgraph with state transformation.

In [ ]:
def create_subgraph_node(subgraph, state_in, state_out):
    """
    Wrap a subgraph as a node in parent graph.
    
    Args:
        subgraph: Compiled subgraph
        state_in: Transform parent state to subgraph input
        state_out: Transform subgraph output to parent updates
    
    Returns:
        Node function for parent graph
    """
    def node(state):
        # Transform state
        subgraph_input = state_in(state)
        
        # Run subgraph
        subgraph_output = subgraph.invoke(subgraph_input)
        
        # Transform output
        return state_out(subgraph_output, state)
    
    return node


# Create the wrapped node
summary_node = create_subgraph_node(
    summary_graph,
    summary_state_in,
    summary_state_out,
)

print("Subgraph wrapped as node!")

## Step 6: Build Parent Graph with Embedded Subgraph

In [ ]:
from langchain_core.messages import AIMessage

def validate_node(state: DocumentState) -> dict:
    """Validate document has content."""
    if not state.get("document_content"):
        return {"messages": [AIMessage(content="No document provided.")]}
    return {"messages": [AIMessage(content="Document validated.")]}


def finalize_node(state: DocumentState) -> dict:
    """Mark processing complete."""
    return {
        "processed": True,
        "messages": [AIMessage(content=f"Summary complete: {state['document_summary'][:100]}...")],
    }


# Build parent graph
parent_workflow = StateGraph(DocumentState)

# Add nodes
parent_workflow.add_node("validate", validate_node)
parent_workflow.add_node("summarize", summary_node)  # Our wrapped subgraph!
parent_workflow.add_node("finalize", finalize_node)

# Add edges
parent_workflow.add_edge(START, "validate")
parent_workflow.add_edge("validate", "summarize")
parent_workflow.add_edge("summarize", "finalize")
parent_workflow.add_edge("finalize", END)

parent_graph = parent_workflow.compile()

print("Parent graph with embedded subgraph compiled!")
print("\nFlow: validate -> summarize (subgraph) -> finalize")

## Step 7: Run the Composed System

In [ ]:
document = """Machine learning is a subset of artificial intelligence that enables 
systems to learn and improve from experience without being explicitly programmed. 
It focuses on developing algorithms that can access data and use it to learn for 
themselves. The process begins with observations or data, such as examples, direct 
experience, or instruction, to look for patterns in data and make better decisions 
in the future. The primary aim is to allow computers to learn automatically without 
human intervention and adjust actions accordingly."""

result = parent_graph.invoke({
    "messages": [],
    "document_content": document,
    "document_summary": "",
    "processed": False,
})

print("Result:")
print("="*50)
print(f"Processed: {result['processed']}")
print(f"\nSummary:\n{result['document_summary']}")

## Step 8: Using the Built-in Module

The `langgraph_ollama_local.patterns.subgraphs` module provides these utilities:

In [ ]:
from langgraph_ollama_local.patterns.subgraphs import (
    create_subgraph_node,
    field_mapper_in,
    field_mapper_out,
)

# Use field mappers for simple transformations
state_in = field_mapper_in(
    ("document_content", "text"),  # parent_field -> subgraph_field
)

state_out = field_mapper_out(
    ("summary", "document_summary"),  # subgraph_field -> parent_field
)

# Create node with module function
summary_node_v2 = create_subgraph_node(summary_graph, state_in, state_out)

print("Using module utilities!")
print("field_mapper_in: creates state_in from field mappings")
print("field_mapper_out: creates state_out from field mappings")

## Step 9: Chaining Multiple Subgraphs

You can chain subgraphs for sequential processing pipelines.

In [ ]:
# Create another subgraph: sentiment analysis
class SentimentState(TypedDict):
    text: str
    sentiment: str
    confidence: float


def sentiment_node(state: SentimentState) -> dict:
    response = llm.invoke([
        SystemMessage(content="Analyze the sentiment. Respond with: POSITIVE, NEGATIVE, or NEUTRAL"),
        HumanMessage(content=state["text"]),
    ])
    sentiment = response.content.strip().upper()
    if "POSITIVE" in sentiment:
        return {"sentiment": "POSITIVE", "confidence": 0.8}
    elif "NEGATIVE" in sentiment:
        return {"sentiment": "NEGATIVE", "confidence": 0.8}
    return {"sentiment": "NEUTRAL", "confidence": 0.6}


sentiment_workflow = StateGraph(SentimentState)
sentiment_workflow.add_node("analyze", sentiment_node)
sentiment_workflow.add_edge(START, "analyze")
sentiment_workflow.add_edge("analyze", END)
sentiment_graph = sentiment_workflow.compile()

print("Sentiment subgraph created!")

In [ ]:
from langgraph_ollama_local.patterns.subgraphs import chain_subgraphs

# Extended parent state
class FullAnalysisState(TypedDict):
    document_content: str
    document_summary: str
    document_sentiment: str


# Chain: summarize then analyze sentiment
chained_node = chain_subgraphs([
    (
        summary_graph,
        lambda s: {"text": s["document_content"], "summary": "", "word_count": 0},
        lambda out, s: {"document_summary": out["summary"]},
    ),
    (
        sentiment_graph,
        lambda s: {"text": s.get("document_summary", s["document_content"]), "sentiment": "", "confidence": 0},
        lambda out, s: {"document_sentiment": out["sentiment"]},
    ),
])

# Build graph with chained subgraphs
analysis_workflow = StateGraph(FullAnalysisState)
analysis_workflow.add_node("analyze", chained_node)
analysis_workflow.add_edge(START, "analyze")
analysis_workflow.add_edge("analyze", END)
analysis_graph = analysis_workflow.compile()

# Test
result = analysis_graph.invoke({
    "document_content": "I absolutely love this product! It exceeded all my expectations and works perfectly.",
    "document_summary": "",
    "document_sentiment": "",
})

print(f"Summary: {result['document_summary']}")
print(f"Sentiment: {result['document_sentiment']}")

## Step 10: Conditional Subgraphs

Run different subgraphs based on conditions.

In [ ]:
from langgraph_ollama_local.patterns.subgraphs import conditional_subgraph

# Create a "short summary" subgraph
def short_summary_node(state):
    response = llm.invoke([
        SystemMessage(content="Summarize in exactly one sentence."),
        HumanMessage(content=state["text"]),
    ])
    return {"summary": response.content, "word_count": len(response.content.split())}

short_workflow = StateGraph(SummaryState)
short_workflow.add_node("summarize", short_summary_node)
short_workflow.add_edge(START, "summarize")
short_workflow.add_edge("summarize", END)
short_graph = short_workflow.compile()


# Condition: use short summary for texts under 100 words
def is_short_text(state):
    text = state.get("document_content", "")
    return len(text.split()) < 100


# Conditional node
conditional_summary_node = conditional_subgraph(
    is_short_text,
    # True: use short summary
    (short_graph, summary_state_in, summary_state_out),
    # False: use full summary
    (summary_graph, summary_state_in, summary_state_out),
)

print("Conditional subgraph node created!")
print("Short texts (<100 words) -> one-sentence summary")
print("Long texts (>=100 words) -> full summary")

## Complete Code

In [ ]:
# === Complete Subgraph Patterns Implementation ===

from typing import Annotated, Callable
from typing_extensions import TypedDict

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langgraph_ollama_local import LocalAgentConfig


# === Subgraph Node Wrapper ===
def create_subgraph_node(subgraph, state_in, state_out):
    """Wrap subgraph as parent graph node."""
    def node(state):
        sub_input = state_in(state)
        sub_output = subgraph.invoke(sub_input)
        return state_out(sub_output, state)
    return node


# === Field Mappers ===
def field_mapper_in(*mappings):
    """Create state_in from (parent_field, sub_field) mappings."""
    def state_in(parent):
        return {sub: parent.get(par, "") for par, sub in mappings}
    return state_in

def field_mapper_out(*mappings):
    """Create state_out from (sub_field, parent_field) mappings."""
    def state_out(sub, parent):
        return {par: sub.get(sf, "") for sf, par in mappings}
    return state_out


# === Chain Subgraphs ===
def chain_subgraphs(subgraphs):
    """Chain (subgraph, state_in, state_out) tuples."""
    def chained(state):
        current = state.copy()
        for sg, sin, sout in subgraphs:
            out = sg.invoke(sin(current))
            current.update(sout(out, current))
        return {k: v for k, v in current.items() if k not in state or state[k] != v}
    return chained


# === Conditional Subgraph ===
def conditional_subgraph(condition, true_sg, false_sg=None):
    """Run subgraph based on condition."""
    def conditional(state):
        if condition(state):
            sg, sin, sout = true_sg
        elif false_sg:
            sg, sin, sout = false_sg
        else:
            return {}
        return sout(sg.invoke(sin(state)), state)
    return conditional


# === Example Usage ===
if __name__ == "__main__":
    cfg = LocalAgentConfig()
    llm = ChatOllama(model=cfg.ollama.model, base_url=cfg.ollama.base_url, temperature=0)
    
    # Create subgraph
    class SubState(TypedDict):
        text: str
        result: str
    
    def process(state):
        r = llm.invoke([SystemMessage(content="Summarize:"), HumanMessage(content=state["text"])])
        return {"result": r.content}
    
    sub = StateGraph(SubState)
    sub.add_node("process", process)
    sub.add_edge(START, "process")
    sub.add_edge("process", END)
    subgraph = sub.compile()
    
    # Create parent with subgraph
    class ParentState(TypedDict):
        input_text: str
        output_summary: str
    
    node = create_subgraph_node(
        subgraph,
        field_mapper_in(("input_text", "text")),
        field_mapper_out(("result", "output_summary")),
    )
    
    parent = StateGraph(ParentState)
    parent.add_node("summarize", node)
    parent.add_edge(START, "summarize")
    parent.add_edge("summarize", END)
    graph = parent.compile()
    
    result = graph.invoke({"input_text": "Python is great for ML.", "output_summary": ""})
    print(result["output_summary"])

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Subgraph** | Complete, compiled graph used as component |
| **State Transformation** | Functions to convert between state schemas |
| **state_in** | Parent state → Subgraph input |
| **state_out** | Subgraph output → Parent state updates |
| **Field Mappers** | Simple transformation for field renaming |
| **Chain** | Sequential subgraph execution |
| **Conditional** | Choose subgraph based on state |

## When to Use Subgraphs

| Use Subgraphs When | Don't Use When |
|-------------------|----------------|
| Reusing logic across graphs | Single-use logic |
| Different state schemas needed | Same state schema |
| Testing components independently | Tightly coupled logic |
| Swapping implementations | Fixed implementation |

## Summary

You've learned the core patterns for building modular, composable LangGraph systems:

1. **Multi-Agent Collaboration** (Tutorial 14): Supervisor coordinates specialists
2. **Hierarchical Teams** (Tutorial 15): Nested team structures
3. **Subgraph Patterns** (Tutorial 16): Composable, reusable components

These patterns can be combined to build sophisticated agent systems that are maintainable, testable, and scalable.